In [ ]:
pip install delta-spark

In [ ]:
pip install sparksql-magic

In [1]:
%load_ext sparksql_magic

In [ ]:
spark.stop()

In [2]:
from pyspark.sql.types import  *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from datetime import datetime, date
from pyspark.sql import Row
from delta import *
from delta.tables import *
import pyspark
import tabulate

In [3]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '4g'), 
    ('spark.executor.cores', '2'), 
    ('spark.cores.max', '6'),
    ('spark.driver.memory','1g'),
    ('spark.executor.instances', '1'),
    ('spark.dynamicAllocation.enabled', 'true'),
    ('spark.dynamicAllocation.shuffleTracking.enabled', 'true'),
    ('spark.dynamicAllocation.executorIdleTimeout', '60s'),
    ('spark.dynamicAllocation.minExecutors', '0'),
    ('spark.dynamicAllocation.maxExecutors', '2'),
    ('spark.dynamicAllocation.initialExecutors', '1'),
    ('spark.dynamicAllocation.executorAllocationRatio', '1'),
    ('spark.sql.repl.eagerEval.enabled', 'true'),
    ('spark.databricks.delta.retentionDurationCheck.enabled', 'false')
])

In [4]:
builder = pyspark.sql.SparkSession.builder \
    .appName("Delta-TABLE-MGMT") \
    .master("spark://172.23.149.212:7077") \
    .config(conf=config) \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/06/04 20:10:36 WARN Utils: Your hostname, algorand-druid-and-spark resolves to a loopback address: 127.0.0.1; using 172.23.149.212 instead (on interface ens3)
22/06/04 20:10:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-52e79470-bb95-4843-b12f-2268c0a8025f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 185ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

In [ ]:
# table names
# delta.`/mnt/delta/bronze/algod.indexer.public.account.test`
# delta.`/mnt/delta/bronze/algod.indexer.public.account`
# delta.`/mnt/delta/bronze/algod_indexer_public_txn_flat`

In [ ]:
%%sparksql
SELECT max(round) FROM delta.`/mnt/delta/bronze/algod_indexer_public_block_header_flat`

In [ ]:
%%sparksql
SELECT round FROM delta.`/mnt/delta/bronze/algod_indexer_public_block_header_flat` LIMIT 10

In [ ]:
%%sparksql
SELECT "round", count("realtime") FROM delta.`/mnt/delta/bronze/algod_indexer_public_block_header_flat` GROUP BY ROUND HAVING COUNT("realtime") 

In [ ]:
%%sparksql
SHOW TBLPROPERTIES delta.`/mnt/delta/bronze/algod_indexer_public_asset_flat` ('delta.logRetentionDuration');

In [ ]:
%%sparksql
SHOW TBLPROPERTIES delta.`/mnt/delta/bronze/algod_indexer_public_asset_flat`;

In [ ]:
%%sparksql
ALTER TABLE delta.`/mnt/delta/bronze/algod_indexer_public_asset_flat` SET TBLPROPERTIES ('delta.logRetentionDuration'='interval 24 hours', 'delta.deletedFileRetentionDuration'='interval 1 hours')

In [ ]:
%%sparksql
VACUUM delta.`/mnt/delta/bronze/algod_indexer_public_asset_flat` RETAIN 1 HOURS

In [ ]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/mnt/delta/bronze/algod_indexer_public_asset_flat")

fullHistoryDF = deltaTable.history()   # get the full history of the table

In [ ]:
deltaTable.vacuum(1)

In [ ]:
%%sparksql
DESCRIBE DETAIL delta.`/mnt/delta/bronze/algod_indexer_public_txn_flat`

In [ ]:
%%sparksql
OPTIMIZE "/mnt/delta/bronze/algod_indexer_public_txn_flat"

In [ ]:
%%sparksql
CREATE TABLE default.people10m USING DELTA LOCATION '/mnt/delta/bronze/people-10m'